In [20]:
!pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-6bijgzjk
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-6bijgzjk
  Resolved https://github.com/openai/whisper.git to commit 248b6cb124225dd263bb9bd32d060b6517e067f8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [41]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [22]:
# !wget https://zenodo.org/record/6342622/files/v2.0.darija.tar.gz?download=1

In [23]:
# !tar -xf /content/v2.0.darija.tar.gz?download=1 -C /content/gdrive/MyDrive

In [24]:
model_name = "openai/whisper-small"

In [25]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.3 MB/s eta 0:00:00


In [36]:
 !ls {directory}

ls: cannot access '/content/gdrive/MyDrive/Colab': Transport endpoint is not connected
ls: cannot access 'Notebooks/darija/texts/': No such file or directory


In [51]:
import os
import re
import pandas as pd
import librosa
# define the directory where your files are located
directory = "/content/gdrive/MyDrive/Colab Notebooks/darija/"
data = pd.read_csv(directory + 'texts/train.csv',sep = '\t')
# print(data.shape)
# get a list of all the WAV files in the directory
wav_files = [file for file in os.listdir(directory+ '/wavs') if file.endswith('.wav')]
# print(wav_files)
for key,row in data.iterrows():
  if not row['wav'] in wav_files:
    data = data.drop(index=key,axis=0) 
print(data.__len__())


2441


In [52]:
from datasets import load_dataset
from datasets import Dataset
# dataset = load_dataset(, data_files="/content/gdrive/MyDrive/darija/texts/train.csv")
dataset = Dataset.from_pandas(data)

In [29]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [53]:
dataset

Dataset({
    features: ['wav', 'words', 'duration', '__index_level_0__'],
    num_rows: 2441
})

In [54]:
dataset[0]

{'wav': 'clipping-306514b0-d498-41ff-b6ef-333cc36045d8.wav',
 'words': 'النار سمعت واحد قال لصاحبه رخص يكون عندنا شويه التراب',
 'duration': 5.934172335600908,
 '__index_level_0__': 0}

In [56]:
from datasets import Audio

In [57]:
audio_dataset = dataset.cast_column("wav", Audio())


In [58]:
%cd /content/gdrive/MyDrive/'Colab Notebooks'/darija/wavs/

/content/gdrive/.shortcut-targets-by-id/1_vsMXjauFr0zXQ-Q9MMYFmt3nPqKPjQ0/darija/wavs


In [59]:
audio_dataset

Dataset({
    features: ['wav', 'words', 'duration', '__index_level_0__'],
    num_rows: 2441
})

In [60]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.0 MB/s eta 0:00:00


In [61]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Arabic", task="transcribe")

In [62]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

do_lower_case = False
do_remove_punctuation = False

normalizer = BasicTextNormalizer()

In [63]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_name, language="Arabic", task="transcribe")

In [64]:
def prepare_dataset(batch):
    # load and (possibly) resample wav data to 16kHz
    wav = batch["wav"]
    # compute log-Mel input features from input wav array 
    batch["input_features"] = processor.feature_extractor(wav["array"], sampling_rate=wav["sampling_rate"]).input_features[0]
    # compute input length of wav sample in seconds
    batch["input_length"] = len(wav["array"]) / wav["sampling_rate"]
    
    # optional pre-processing steps
    transcription = batch["words"]
    if do_lower_case:
        transcription = transcription.lower()
    if do_remove_punctuation:
        transcription = normalizer(transcription).strip()
    
    # encode target text to label ids
    batch["labels"] = processor.tokenizer(transcription).input_ids
    return batch

In [65]:
audio_dataset

Dataset({
    features: ['wav', 'words', 'duration', '__index_level_0__'],
    num_rows: 2441
})

In [66]:
audio_dataset[0]

{'wav': {'path': 'clipping-306514b0-d498-41ff-b6ef-333cc36045d8.wav',
  'array': array([-0.00048828, -0.00048828, -0.00048828, ..., -0.00231934,
         -0.00134277, -0.00180054]),
  'sampling_rate': 16000},
 'words': 'النار سمعت واحد قال لصاحبه رخص يكون عندنا شويه التراب',
 'duration': 5.934172335600908,
 '__index_level_0__': 0}

In [67]:
min_dataset = audio_dataset.select(range(20))

In [68]:
min_dataset = min_dataset.map(prepare_dataset, remove_columns=min_dataset.column_names, num_proc=12)


Map (num_proc=12):   0%|          | 0/20 [00:00<?, ? examples/s]

In [69]:
max_input_length = 30.0

def is_audio_in_length_range(length):
    return length < max_input_length

In [70]:
min_dataset = min_dataset.filter(
    is_audio_in_length_range,
    input_columns=["input_length"],)

Filter:   0%|          | 0/20 [00:00<?, ? examples/s]

In [71]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [72]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [73]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.1 MB/s eta 0:00:00


In [74]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.1 MB/s eta 0:00:00


In [75]:
import evaluate
metric = evaluate.load("wer")


In [76]:
# evaluate with the 'normalised' WER
do_normalize_eval = True

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    if do_normalize_eval:
        pred_str = [normalizer(pred) for pred in pred_str]
        label_str = [normalizer(label) for label in label_str]

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [77]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name)

In [78]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.use_cache = False

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="ADHD",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=5,
    max_steps=50,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=6,
    eval_steps=6,
    logging_steps=5,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
!_huggingface-cli login

In [ ]:
data_eval = pd.read_csv('/content/test.csv',sep = '\t')
# get a list of all the WAV files in the directory
wav_files = [file for file in os.listdir(directory) if file.endswith('.wav')]
for key,row in data_eval.iterrows():
  if not row['wav'] in wav_files:
    data_eval= data_eval.drop(index=key,axis=0) 
# dataset = load_dataset(, data_files="/content/gdrive/MyDrive/darija/texts/train.csv")
dataset = Dataset.from_pandas(data_eval)
eval_data = dataset.cast_column("wav", Audio())
mini_dataset = eval_data.select(range(20))
mini_dataset = mini_dataset.map(prepare_dataset, remove_columns=mini_dataset.column_names, num_proc=12)
mini_dataset = mini_dataset.filter(
    is_audio_in_length_range, 
    input_columns=["input_length"],)



In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=min_dataset,
    eval_dataset=mini_dataset.select(range(20)),
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
processor.save_pretrained(training_args.output_dir)

In [ ]:
trainer.train()

In [ ]:
kwargs = {
    # "dataset_tags": "mozilla-foundation/common_voice_11_0",
    # "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "language": "ar",
    "model_name": "Whisper Small ar - adhd",  # a 'pretty' name for your model
    "finetuned_from": model_name,
    "tasks": "automatic-speech-recognition",
    # "tags": "whisper-event",
}

In [ ]:
processor.tokenizer

In [ ]:
model

In [ ]:
from transformers import pipeline
from huggingface_hub import model_info

MODEL_NAME = "arbml/whisper-medium-ar" #this always needs to stay in line 8 :D sorry for the hackiness
lang = "ar"

pipe = pipeline(
    tokenizer = processor.tokenizer,
    feature_extractor = processor.feature_extractor,
    task="automatic-speech-recognition",
    model=trainer.model,
    chunk_length_s=30,
    device="cuda:0",
)


In [91]:
text = pipe('/content/gdrive/MyDrive/Colab Notebooks/darija/wavs/1b43cac2-7d9c-463c-8e36-7f2fec9d2c6a.wav')["text"]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [92]:
text

'خيره اللي جزيكم بالخير و'

In [80]:
DIR = '/content/gdrive/MyDrive/Colab Notebooks/darija/wavs/ADHD'

In [81]:
model = WhisperForConditionalGeneration.from_pretrained(DIR)

In [100]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [110]:
import gradio as gr

def transcribe(audio):
    text = pipe(audio)["text"]
    return text



gr.Interface(
    fn=transcribe, 
    inputs=gr.Audio(source="microphone", type="filepath"), 
    outputs="text").launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>